# **Descripción del proyecto**

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes
utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el
comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:
Smart o Ultra.
Para esta tarea de clasificación debo crear un modelo que escoja el plan correcto.
Para entrenar mi modelo, utilizaré los datos de comportamiento del usuario
del curso sobre Análisis estadísticos de datos. Como ya hice el paso de procesar los
datos, puedo lanzarme directo a crear el modelo.
Desarrollaré un modelo con la mayor exactitud posible. En este proyecto, debe superar el umbral de
exactitud de 0.75.

---


#  Flujo de trabajo del proyecto

En este proyecto voy a construir y evaluar distintos modelos de **clasificación** para predecir si un cliente pertenece a la tarifa *Ultra o Smart* a partir de su uso de llamadas, minutos, mensajes y datos móviles.

El flujo de trabajo que voy a seguir es el siguiente:

1. **Importación de librerías**
   Inicio cargando todas las herramientas necesarias: `pandas` para la manipulación de datos, módulos de `Scikit-Learn` para la construcción de modelos de clasificación y la evaluación de métricas y `joblib` para guardar el modelo final.

2. **Carga y exploración de datos (EDA rápido)**
   Importo el dataset y realizo una exploración inicial para verificar que los datos sean correctos: ausencia de valores nulos, tipos adecuados de variables y distribución general de las características.

3. **Definición de variables**
   Separo el dataset en:

   * **Features (X):** `calls`, `minutes`, `messages`, `mb_used`.
   * **Target (y):** `is_Ultra`.

4. **División en entrenamiento y prueba (sin conjunto de validación ya que se utiliza GridSearchCV)**
   Divido los datos en conjuntos de **entrenamiento** y **prueba**, manteniendo la proporción de clases para garantizar una evaluación justa.

5. **Entrenamiento con GridSearchCV**
   Utilizo la búsqueda de hiperparámetros con validación cruzada (`GridSearchCV`) para ajustar tres modelos distintos:

   * **Decision Tree Classifier**
   * **Random Forest Classifier**
   * **Logistic Regression** (con escalado de features)

6. **Selección del mejor modelo**
   Identifico los mejores hiperparámetros para cada modelo y evalúo el rendimiento en el conjunto de prueba. Finalmente, selecciono el modelo con la mayor exactitud (*accuracy*) como el más adecuado para esta tarea y lo guardo por si se quiere utilizar más adelante.

7. **Prueba de cordura**
   Realizo una prueba de cordura para asegurarme de que el modelo seleccionado generaliza bien a datos no vistos. Esto implica comparar el modelo entrenado contra un modelo tonto (baseline) que no aprende nada de los datos.

      - Si el modelo tiene un rendimiento similar o peor que el baseline → no está aprendiendo nada útil.

      - Si el modelo supera claramente al baseline →  demuestra que sí captura patrones.

8. **Conclusión**
   Resumo los hallazgos clave, el rendimiento del modelo seleccionado.

## **Importación de librerías**
   Inicio cargando todas las herramientas necesarias: `pandas` para la manipulación de datos, módulos de `Scikit-Learn` para la construcción de modelos de clasificación y la evaluación de métricas y `joblib` para guardar el modelo final.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
# Librerías para modelado
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
# Librerías para evaluación
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
# Librería para guardar el mejor modelo
from joblib import dump, load
# Establecer semilla para reproducibilidad
RANDOM_STATE = 12345

## **Carga y exploración de datos (EDA rápido)**
   Importo el dataset y realizo una exploración inicial para verificar que los datos sean correctos: ausencia de valores nulos, tipos adecuados de variables y distribución general de las características.

In [2]:
# Cargar los datos
df = pd.read_csv(r'..\data\users_behavior.csv')

In [3]:
# Pequeña exploración de los datos
display(df.head())
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

## **Definición de variables**
   Separo el dataset en:

   * **Features (X):** `calls`, `minutes`, `messages`, `mb_used`.
   * **Target (y):** `is_Ultra`.

In [4]:
# Dividir los datos en features y target
X = df.drop(columns=['is_ultra'])
y = df['is_ultra']
# Comprobar si tienen la misma longitud
print(f'Features: {X.shape[0]}, Target: {y.shape[0]}')

Features: 3214, Target: 3214


## **División en entrenamiento y prueba (sin conjunto de validación ya que se utiliza GridSearchCV)**
   Divido los datos en conjuntos de **entrenamiento** y **prueba**, manteniendo la proporción de clases para garantizar una evaluación justa.

In [5]:
# Dividir los datos para entrenamiento y prueba ya que trabajaré con gridsearch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE)
# Comprobar las dimensiones de los conjuntos
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (2571, 4), X_test: (643, 4), y_train: (2571,), y_test: (643,)


## **Entrenamiento con GridSearchCV**
   Utilizo la búsqueda de hiperparámetros con validación cruzada (`GridSearchCV`) para ajustar tres modelos distintos:

   * **Decision Tree Classifier**
   * **Random Forest Classifier**
   * **Logistic Regression** (con escalado de features)

In [6]:
# Configurar la validación cruzada estratificada con StratifiedKFold en 5 pliegues
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [7]:
# Empezaré con un Decision Tree
# Definir el modelo
tree_model = DecisionTreeClassifier(random_state=RANDOM_STATE)
# Definir los hiperparámetros a probar
tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 11)),   
    'min_samples_split': list(range(2, 11,2)),    
    'min_samples_leaf': list(range(1, 11,2))      
}

# Configurar GridSearchCV
grid_tree = GridSearchCV(estimator=tree_model, param_grid=tree_params, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# Entrenar el modelo con los datos de entrenamiento
grid_tree.fit(X_train, y_train)
# Obtener el mejor modelo y mejores hiperparámetros
best_tree = grid_tree.best_estimator_
best_tree_params = grid_tree.best_params_
print(f'Mejores hiperparámetros para Decision Tree: {best_tree_params}')
print(f'Mejor score de validación cruzada para Decision Tree: {grid_tree.best_score_:.3f}')

Mejores hiperparámetros para Decision Tree: {'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 9, 'min_samples_split': 2}
Mejor score de validación cruzada para Decision Tree: 0.802


In [8]:
# Probar el mejor modelo en el conjunto de prueba
pred_tree = best_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, pred_tree)
print(f'Accuracy en el conjunto de prueba para Decision Tree: {accuracy_tree:.3f}')

Accuracy en el conjunto de prueba para Decision Tree: 0.782


In [9]:
# Continuamos con Random Forest
# Definir el modelo
forest_model = RandomForestClassifier(random_state=RANDOM_STATE)
# Definir los hiperparámetros a probar
forest_params = {
    'n_estimators': list(range(10, 51, 10)),
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 11)),   
    'min_samples_split':list(range(2,11,2)),   
    'min_samples_leaf': list(range(1,8,2))   
}
# Configurar GridSearchCV
grid_forest = GridSearchCV(estimator=forest_model, param_grid=forest_params, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# Entrenar el modelo con los datos de entrenamiento
grid_forest.fit(X_train, y_train)
# Obtener el mejor modelo y mejores hiperparámetros
best_forest = grid_forest.best_estimator_
best_forest_params = grid_forest.best_params_
print(f'Mejores hiperparámetros para Random Forest: {best_forest_params}')
print(f'Mejor score de validación cruzada para Random Forest: {grid_forest.best_score_:.3f}')

Mejores hiperparámetros para Random Forest: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 30}
Mejor score de validación cruzada para Random Forest: 0.817


In [10]:
# Evaluar el mejor modelo en el conjunto de prueba
pred_forest = best_forest.predict(X_test)
accuracy_forest = accuracy_score(y_test, pred_forest)
print(f'Accuracy en el conjunto de prueba para Random Forest: {accuracy_forest:.3f}')

Accuracy en el conjunto de prueba para Random Forest: 0.795


In [11]:
# Por ultimo, probaré con Regresión Logística
# Definir el modelo con un pipeline que incluya escalado ya que los modelos lineales lo requieren
log_model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))
])
# Definir los hiperparámetros a probar
param_grid_log = [
    {   
        "logistic__solver": ["liblinear"],
        "logistic__penalty": ["l1", "l2"],
        "logistic__C": [0.01, 0.1, 1, 10, 100]
    }]
# Configurar GridSearchCV
grid_log = GridSearchCV(estimator=log_model, param_grid=param_grid_log, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# Entrenar el modelo con los datos de entrenamiento
grid_log.fit(X_train, y_train)
# Obtener el mejor modelo y mejores hiperparámetros
best_log = grid_log.best_estimator_
best_log_params = grid_log.best_params_
print(f'Mejores hiperparámetros para Regresión Logística: {best_log_params}')
print(f'Mejor score de validación cruzada para Regresión Logística: {grid_log.best_score_:.3f}')

Mejores hiperparámetros para Regresión Logística: {'logistic__C': 0.1, 'logistic__penalty': 'l2', 'logistic__solver': 'liblinear'}
Mejor score de validación cruzada para Regresión Logística: 0.748


In [12]:
# Evaluar el mejor modelo en el conjunto de prueba
pred_log = best_log.predict(X_test)
accuracy_log = accuracy_score(y_test, pred_log)
print(f'Accuracy en el conjunto de prueba para Regresión Logística: {accuracy_log:.3f}')

Accuracy en el conjunto de prueba para Regresión Logística: 0.757


## **Selección del mejor modelo**
   Ahora que ya identifiqué los mejores hiperparámetros para cada modelo y evalué el rendimiento en el conjunto de prueba. Finalmente, selecciono el modelo con la mayor exactitud (*accuracy*) como el más adecuado para esta tarea y lo guardé por si se quiere utilizar más adelante.

In [13]:
# Conclusión: El mejor modelo es el que tiene mayor accuracy en el conjunto de prueba
best_model = None
if accuracy_tree >= accuracy_forest and accuracy_tree >= accuracy_log:
    best_model = best_tree
    print(f"El mejor modelo es Decision Tree con exactitud: {(accuracy_tree)*100:.2f}%")
elif accuracy_forest >= accuracy_tree and accuracy_forest >= accuracy_log:
    best_model = best_forest
    print(f"El mejor modelo es Random Forest con exactitud: {(accuracy_forest)*100:.2f}%")
else:
    best_model = best_log
    print(f"El mejor modelo es Regresión Logística con exactitud: {(accuracy_log)*100:.2f}%")

El mejor modelo es Random Forest con exactitud: 79.47%


## Prueba de cordura
 Realizo una prueba de cordura para asegurarme de que el modelo seleccionado generaliza bien a datos no vistos. Esto implica comparar el modelo entrenado contra un modelo tonto (baseline) que no aprende nada de los datos.

    - Si el modelo tiene un rendimiento similar o peor que el baseline → no está aprendiendo nada útil.

    - Si el modelo supera claramente al baseline →  demuestra que sí captura patrones.

In [14]:
# Definir el modelo tonto (baseline)
dummy_model = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
# Entrenar el modelo tonto con los datos de entrenamiento
dummy_model.fit(X_train, y_train)
# Predecir con el modelo tonto en el conjunto de prueba
pred_dummy = dummy_model.predict(X_test)
# Evaluar el modelo tonto en el conjunto de prueba
accuracy_dummy = accuracy_score(y_test, pred_dummy)
print(f'Accuracy del modelo tonto (baseline) en el conjunto de prueba: {accuracy_dummy:.3f}')

Accuracy del modelo tonto (baseline) en el conjunto de prueba: 0.695


In [15]:
# Comparar el mejor modelo con el baseline
if accuracy_forest > accuracy_dummy:
    print("El mejor modelo supera claramente al baseline, demostrando que captura patrones útiles y supera la prueba de cordura.")
    print(f'Accuracy del mejor modelo (Random Forest) en el conjunto de prueba: {accuracy_forest:.3f}')
else:
    print("El mejor modelo no supera al baseline, indicando que no está aprendiendo patrones útiles y falla la prueba de cordura.")

El mejor modelo supera claramente al baseline, demostrando que captura patrones útiles y supera la prueba de cordura.
Accuracy del mejor modelo (Random Forest) en el conjunto de prueba: 0.795


In [17]:
# Guardar el mejor modelo
dump(best_model, r'..\models\best_model.joblib')

['..\\models\\best_model.joblib']

##  Conclusiones

En este proyecto se construyeron y compararon tres modelos de clasificación (**Decision Tree**, **Random Forest** y **Logistic Regression**) con el objetivo de predecir si un cliente pertenece a la tarifa *Ultra o *.

* El **modelo Dummy (baseline)**, que siempre predice la clase mayoritaria, alcanzó una exactitud del **69.5%** en el conjunto de prueba.
* El **mejor modelo encontrado fue Random Forest**, con exactitud del **79.5%** en el conjunto de prueba, superando claramente al baseline y demostrando que captura patrones útiles en los datos.
* El modelo no solo mejora la predicción en un **+10% absoluto** frente al baseline, sino que también confirma que los patrones de uso de llamadas, minutos, mensajes y datos móviles son relevantes para distinguir entre clientes *Ultra* y *Smart*.

###  Reflexión final

* El **árbol de decisión** resultó más simple pero menos preciso, mostrando riesgo de sobreajuste o subajuste según la profundidad con una exactitud del **78.2%**.
* La **regresión logística** fue rápida y fácil de interpretar, pero no alcanzó la precisión del bosque con una exactitud del **75.7%**.
* El **bosque aleatorio** combinó la robustez de varios árboles y logró el mejor equilibrio entre sesgo y varianza, lo que lo convierte en el modelo más adecuado para este problema con una exactitud del **79.5%**.

En conclusión, el **Random Forest Classifier** es el modelo recomendado, ya que supera ampliamente la prueba de cordura y ofrece el mejor rendimiento en el conjunto de prueba, con una exactitud suficiente para ser usado como herramienta predictiva en este escenario.

---